In [8]:
import numpy as np
import pandas as pd

data = pd.read_csv('dataset/in_and_out.csv', encoding='cp949')
data

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202504,1호선,동대문,650,26,11355,2005,10238,7181,18411,...,9939,938,3210,3,321,0,0,0,0,20250503
1,202504,1호선,동묘앞,117,3,2844,819,4206,5444,9689,...,4897,120,1996,3,267,0,0,0,0,20250503
2,202504,1호선,서울역,712,48,10654,9668,29272,64058,97455,...,29252,5212,8018,10,262,0,0,0,0,20250503
3,202504,1호선,시청,124,5,2267,5900,3888,28761,7841,...,3763,606,1015,0,15,0,0,0,0,20250503
4,202504,1호선,신설동,381,18,9393,2330,10420,9020,24157,...,9416,455,3039,0,0,1,2,0,0,20250503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14857,202305,중앙선,원덕,95,0,394,6,740,182,885,...,328,3,80,0,0,0,0,0,0,20230603
14858,202305,중앙선,중랑,153,5,6886,593,10126,2329,24074,...,5311,34,486,0,0,0,0,0,0,20230603
14859,202305,중앙선,지평,0,0,0,0,4,2,384,...,0,0,0,0,0,0,0,0,0,20230603
14860,202305,중앙선,팔당,0,0,259,78,726,728,1658,...,202,0,0,0,0,0,0,0,0,20230603


In [12]:
from tableone import TableOne

# 1. 컬럼 공백 제거
data.columns = data.columns.str.strip()
# - 데이터프레임의 모든 컬럼명에서 앞뒤 공백을 제거합니다.
# - 컬럼명 오타, 불필요한 공백 등으로 인한 오류를 예방합니다.

# 2. 승차/하차 시간대 컬럼만 추출 (총 인원 컬럼 제외!)
승차_cols = [col for col in data.columns if '승차' in col and '총' not in col]
# - '승차'가 포함되고 '총'이 포함되지 않은 컬럼만 추출하여 시간대별 승차인원만 선택합니다.
하차_cols = [col for col in data.columns if '하차' in col and '총' not in col]
# - '하차'가 포함되고 '총'이 포함되지 않은 컬럼만 추출하여 시간대별 하차인원만 선택합니다.
numeric_cols = 승차_cols + 하차_cols
# - 분석에 사용할 모든 시간대별 승차/하차 컬럼 리스트를 만듭니다.

# 3. 총 승하차 인원 계산
data['총승차인원'] = data[승차_cols].sum(axis=1, skipna=True)
# - 각 행(관측치)별로 시간대별 승차인원 합계를 계산하여 '총승차인원' 컬럼에 저장합니다.
data['총하차인원'] = data[하차_cols].sum(axis=1, skipna=True)
# - 각 행별 시간대별 하차인원 합계를 '총하차인원' 컬럼에 저장합니다.

# 4. 사용월 문자열 변환
data['사용월'] = data['사용월'].astype(str)
# - '사용월' 컬럼을 문자열로 변환합니다.
# - TableOne에서 범주형 변수로 인식시키기 위함입니다.

# 5. TableOne 생성
columns = ['사용월', '호선명'] + numeric_cols + ['총승차인원', '총하차인원']
# - TableOne에 사용할 컬럼명 리스트를 생성합니다.
table = TableOne(
    data=data,  # 분석 대상 데이터프레임
    columns=columns,  # 분석할 변수 전체(범주형+연속형)
    categorical=['사용월', '호선명'],  # 범주형 변수: 사용월, 호선명
    nonnormal=numeric_cols + ['총승차인원', '총하차인원'],  # 비정규분포(중앙값, 사분위수 등)로 처리할 연속형 변수
    pval=False  # 그룹 간 p-value(통계적 유의성) 계산은 하지 않음
)

# 6. 출력
print(table.tabulate(tablefmt="github"))
# - TableOne 결과를 GitHub 스타일 표로 출력합니다.
# - 월별, 노선별, 시간대별 승하차 인원 및 총합에 대한 기술통계(중앙값, 사분위수 등)를 한눈에 볼 수 있습니다.



|                                    |                | Missing   | Overall                      |
|------------------------------------|----------------|-----------|------------------------------|
| n                                  |                |           | 14862                        |
| 사용월, n (%)                      | 202305         |           | 614 (4.1)                    |
|                                    | 202306         |           | 614 (4.1)                    |
|                                    | 202307         |           | 616 (4.1)                    |
|                                    | 202308         |           | 616 (4.1)                    |
|                                    | 202309         |           | 616 (4.1)                    |
|                                    | 202310         |           | 617 (4.2)                    |
|                                    | 202311         |           | 617 (4.2)                    |
|            

In [11]:

# 컬럼명 공백 제거
data.columns = data.columns.str.strip()
# - 데이터프레임의 모든 컬럼명에서 앞뒤 공백을 제거하여, 컬럼명 오타 및 불일치 문제 방지

# 시간대별 승차/하차 컬럼 추출 (총 인원 컬럼 제외)
승차_cols = [col for col in data.columns if '승차' in col and '총' not in col]
# - '승차'가 포함되고 '총'이 포함되지 않은 컬럼만 추출 (시간대별 승차인원만 선택)
하차_cols = [col for col in data.columns if '하차' in col and '총' not in col]
# - '하차'가 포함되고 '총'이 포함되지 않은 컬럼만 추출 (시간대별 하차인원만 선택)
numeric_cols = 승차_cols + 하차_cols
# - 분석에 사용할 모든 시간대별 승차/하차 컬럼 리스트 생성

# 총 승하차 인원 계산
data['총승차인원'] = data[승차_cols].sum(axis=1, skipna=True)
# - 각 행(관측치)별로 시간대별 승차인원 합계를 계산하여 '총승차인원' 컬럼에 저장
data['총하차인원'] = data[하차_cols].sum(axis=1, skipna=True)
# - 각 행별 시간대별 하차인원 합계를 '총하차인원' 컬럼에 저장

# 사용월, 호선명 문자열 변환
data['사용월'] = data['사용월'].astype(str)
# - '사용월' 컬럼을 문자열로 변환 (추후 그룹핑 및 문자열 결합을 위해)
data['호선명'] = data['호선명'].astype(str)
# - '호선명' 컬럼도 문자열로 변환

# 그룹 컬럼 생성 (예: '2023-05_1호선')
data['사용월_호선'] = data['사용월'] + '_' + data['호선명']
# - '사용월'과 '호선명'을 결합하여 새로운 그룹핑 컬럼 생성 (월별, 노선별 그룹 분석 목적)

# TableOne 생성 (새 그룹 컬럼으로 그룹화)
table = TableOne(
    data=data,  # 분석 대상 데이터프레임
    columns=numeric_cols + ['총승차인원', '총하차인원'],  # 분석할 연속형 변수(시간대별+총합)
    categorical=[],  # 범주형 변수 없음
    groupby='사용월_호선',  # 그룹핑 기준: '사용월_호선' (월별, 노선별)
    nonnormal=numeric_cols + ['총승차인원', '총하차인원'],  # 모든 변수 비정규분포(중앙값, 사분위수 등)
    pval=True  # 그룹 간 p-value(통계적 유의성) 계산 포함
)

# 출력
print(table.tabulate(tablefmt="github"))
# - TableOne 결과를 GitHub 스타일 표로 출력 (가독성 높음)



|                                    |    | Missing   | Overall                      | 202305_1호선                 | 202305_2호선                  | 202305_3호선                 | 202305_4호선                 | 202305_5호선                 | 202305_6호선                 | 202305_7호선                 | 202305_8호선                 | 202305_9호선                 | 202305_9호선2~3단계          | 202305_경강선             | 202305_경부선                | 202305_경원선                | 202305_경의선               | 202305_경인선                | 202305_경춘선             | 202305_공항철도 1호선        | 202305_과천선                | 202305_분당선                | 202305_수인선               | 202305_신림선               | 202305_안산선                | 202305_우이신설선           | 202305_일산선                | 202305_장항선              | 202305_중앙선              | 202306_1호선                 | 202306_2호선                  | 202306_3호선                 | 202306_4호선                 | 202306_5호선                 | 202306_6호선                 | 202306_7호선          